# Jargon Notes

## Models

***

### General Encoder-Decoder Framework

* __Encoder__ :
    * Reads in source data, produces feature representation in cts space
        * RNN; input sequence of words $\to$ fixed vector that is $\approx$ meaning
        * CNN; input img $\to$ volume that contains higher-level features of img
    * _Idea_ : representation from encoder used by decoder to generate new data
    * [_List of encoders_](https://google.github.io/seq2seq/encoders/)
    
* __Decoder__ :
    * Generative model, conditioned on representation from encoder
        * RNN decoder: learns to generate translation for encoded sequence in another language
    * [_List of decoders_](https://google.github.io/seq2seq/decoders/)
    
* __The model__ :
    * Defines how encoder/decoder put together & how to calculate/minimize loss fn
    * [_List of models_](https://google.github.io/seq2seq/models/)

### Seq2Seq

* [_Source_](https://google.github.io/seq2seq/)

    
    



## Evaluation Metrics